<a href="https://colab.research.google.com/github/choiwoohyck/Keras_Learning/blob/master/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape,test_data.shape)
print(train_targets.shape,test_targets.shape)
print(type(train_data))

(404, 13) (102, 13)
(404,) (102,)
<class 'numpy.ndarray'>


In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std
print(test_data[0])

[ 1.55369355 -0.48361547  1.0283258  -0.25683275  1.03838067  0.23545815
  1.11048828 -0.93976936  1.67588577  1.5652875   0.78447637 -3.48459553
  2.25092074]


In [ ]:
from keras import models
from keras import layers
import tensorflow as tf


from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

def build_model():
  inputs = Input(shape=(13,))
  hidden1 = Dense(64, activation='relu')(inputs)
  hidden2 = Dense(64, activation='relu')(hidden1)
  output = Dense(1)(hidden2)
  model = Model(inputs=inputs, outputs=output)  
  model.compile(optimizer='rmsprop',loss='mse',metrics='mae')
  return model



In [ ]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

num_train_samples = 323 # 404*0.8

validation_data = train_data[num_train_samples:]
validation_targets = train_targets[num_train_samples:]

train_data = train_data[:num_train_samples]
train_targets = train_targets[:num_train_samples]

model = build_model()
model.fit(train_data, train_targets,
          epochs=100, batch_size=16, verbose=0,shuffle=True)
validation_score = model.evaluate(validation_data, validation_targets)

model = build_model()
model.fit(np.concatenate([train_data,validation_data]), np.concatenate([train_targets,validation_targets]),
          epochs=80, batch_size=16, verbose=0,shuffle=True)
validation_score = model.evaluate(test_data, test_targets)

4/4 [==============================] - 0s 2ms/step - loss: 34.0183 - mae: 3.9931


In [ ]:
from sklearn.model_selection import train_test_split

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

# train_test_split
train_data, validation_data,train_targets, validation_targets = train_test_split(train_data, train_targets, test_size=0.2, shuffle=True, random_state=1)

model = build_model()
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0,shuffle=False)
validation_score = model.evaluate(validation_data, validation_targets)

model = build_model()
model.fit(np.concatenate([train_data,validation_data]), np.concatenate([train_targets,validation_targets]),
          epochs=80, batch_size=16, verbose=0,shuffle=False)
validation_score = model.evaluate(test_data, test_targets)

train_data, validation_data,train_targets, validation_targets = train_test_split(train_data, train_targets, test_size=0.2, shuffle=True, random_state=2)

model = build_model()
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0,shuffle=False)
validation_score = model.evaluate(validation_data, validation_targets)

model = build_model()
model.fit(np.concatenate([train_data,validation_data]), np.concatenate([train_targets,validation_targets]),
          epochs=80, batch_size=16, verbose=0,shuffle=False)
validation_score = model.evaluate(test_data, test_targets)

4/4 [==============================] - 0s 1ms/step - loss: 49.5434 - mae: 5.2378


In [ ]:
import numpy as np

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

k = 4
num_val_samples = len(train_data) // k
num_epochs = 80
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0, shuffle = False)
    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

print(np.mean(all_scores))

model = build_model()
model.fit(train_data,train_targets,
          epochs=80, batch_size=16, verbose=0,shuffle=True)
validation_score = model.evaluate(test_data, test_targets)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3
3.7853652238845825
4/4 [==============================] - 0s 2ms/step - loss: 42.2240 - mae: 5.4348


In [ ]:
import numpy as np
import random
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data(seed = random.randint(1,100))


k = 4
num_val_samples = len(train_data) // k
num_epochs = 80
all_scores = []
for j in range(3):
  (train_data, train_targets), (test_data, test_targets) = boston_housing.load_data(seed = random.randint(1,100))
  for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0, shuffle = False)
    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

print(np.mean(all_scores))
model = build_model()
model.fit(train_data,train_targets,
          epochs=80, batch_size=16, verbose=0,shuffle=True)
validation_score = model.evaluate(test_data, test_targets)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3
처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3
처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3
3.4889694849650064
4/4 [==============================] - 0s 2ms/step - loss: 55.4296 - mae: 5.3284


In [ ]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
tmp = [[x,y] for x, y in zip(train_data, train_targets)]
np.random.shuffle(tmp)

train_data = [n[0] for n in tmp]
train_targets = [n[1] for n in tmp]
print(train_data.shape)
print(train_targets.shape)


AttributeError: ignored